In [1]:
%load_ext autoreload
%autoreload 2
from datasets import *
import re

py_data = load_dataset("bigcode/the-stack", split="train", data_files="data/python/train-00000-of-00206.parquet")
java_data = load_dataset("bigcode/the-stack", split="train", data_files="data/java/train-00005-of-00285.parquet")

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/franlucc/.cache/huggingface/datasets/bigcode___parquet/bigcode--the-stack-feef6e400b84213f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/franlucc/.cache/huggingface/datasets/bigcode___parquet/bigcode--the-stack-9f45e66b86f2b01e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [2]:
selected_py_data = [i for i in py_data[:100]["content"] if bool(re.search(r'\d', i))]
selected_java_data = [i for i in java_data[:100]["content"] if bool(re.search(r'int ', i)) and bool(re.search(r'\d', i))]

In [50]:
def crop_int_function(selected_data):
    edited_data = {"python":[]}
    for ex in selected_data[:1]:
        # cut out function with int
        ints = re.findall(r'\n\n *def[\w\W]*?:[\w\W]*?\d[\w\W]*?\n', ex)
        unrolled = [i.split("\n\n")[-1] for i in ints]
        edited_data["python"] += unrolled
    return Dataset.from_dict(edited_data)

py_data = crop_int_function(selected_py_data)

In [53]:
py_data.save_to_disk("data/python_int_functions.parquet")